In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


## Basic Hash Join 

In [3]:
import csv
import time

def read_csv_data(file_path):
    with open(file_path, newline='') as csvfile:
        return list(csv.reader(csvfile))

def hash_join(orders, lineitem):
    hash_table = {order[0]: order for order in orders}

    joined_data = []
    for lineitem in lineitem:
        order_key = lineitem[0]
        if order_key in hash_table:
            joined_data.append(hash_table[order_key] + lineitem)

    return joined_data

# Replace 'orders.csv' and 'lineitems.csv' with your actual file paths
orders_path = '/Users/shaiksaheer/Documents/College_Docs/DataBaseDesign/V3.0.1/dbgen/Data/orders.csv'
lineitems_path = '/Users/shaiksaheer/Documents/College_Docs/DataBaseDesign/V3.0.1/dbgen/Data/lineitem.csv'

# Read in the data
start_time = time.time()
orders_data = read_csv_data(orders_path)
lineitem_data = read_csv_data(lineitem_path)
read_time = time.time() - start_time

# Perform the hash join
start_time = time.time()
joined_data = hash_join(orders_data, lineitem_data)
join_time = time.time() - start_time

# Save the joined data to a new CSV file
output_path = '/Users/shaiksaheer/Documents/College_Docs/DataBaseDesign/V3.0.1/dbgen/Data/joined_data.csv'
with open(output_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in joined_data:
        csvwriter.writerow(row)

# Print out the performance metrics
print(f"Read time: {read_time:.4f} seconds")
print(f"Join time: {join_time:.4f} seconds")
print(f"Total time: {read_time + join_time:.4f} seconds")
print(f"Joined data has been saved to {output_path}")


Read time: 2.5617 seconds
Join time: 5.0788 seconds
Total time: 7.6406 seconds
Joined data has been saved to /Users/shaiksaheer/Documents/College_Docs/DataBaseDesign/V3.0.1/dbgen/Data/joined_data.csv


In [5]:
pip install xxhash

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pyfarmhash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyfarmhash: filename=pyfarmhash-0.3.2-cp310-cp310-macosx_11_0_arm64.whl size=11068 sha256=6a23ee97d818c8ab42850a2796e026894694f3cafeee89663007578daaacbbca
  Stored in directory: /Users/shaiksaheer/Library/Caches/pip/wheels/a3/88/44/d0beeb16b34cbb3d7919ab0db836afe90c656bafd1034b5178
Successfully built pyfarmhash
Note: you may need to restart the kernel to use updated packages.


## Optimized XXhash

In [14]:
import csv
import time
import xxhash

def read_csv_data_in_chunks(file_path, chunk_size=10000):
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        chunk = []
        for i, row in enumerate(reader):
            if i % chunk_size == 0 and i > 0:
                yield chunk
                chunk = []
            chunk.append(row)
        yield chunk

def xxhash_key(value):
    return xxhash.xxh64(value).intdigest()

def hash_join(orders_chunk, lineitem, hash_function):
    hash_table = {}
    for order in orders_chunk:
        hash_key = hash_function(order[0])
        hash_table[hash_key] = order

    joined_data = []
    for item in lineitem:
        hash_key = hash_function(item[0])
        if hash_key in hash_table:
            joined_data.append(hash_table[hash_key] + item)

    return joined_data

orders_path = '/Users/shaiksaheer/Documents/College_Docs/DataBaseDesign/V3.0.1/dbgen/Data/orders.csv'
lineitem_path = '/Users/shaiksaheer/Documents/College_Docs/DataBaseDesign/V3.0.1/dbgen/Data/lineitem.csv'

start_read_time = time.time()
lineitem_data = list(read_csv_data_in_chunks(lineitem_path, chunk_size=10000))[0]
read_time = time.time() - start_read_time

start_join_time = time.time()
joined_data = []
for orders_chunk in read_csv_data_in_chunks(orders_path, chunk_size=10000):
    joined_data.extend(hash_join(orders_chunk, lineitem_data, xxhash_key))
join_time = time.time() - start_join_time

output_path = '/Users/shaiksaheer/Documents/College_Docs/DataBaseDesign/V3.0.1/dbgen/Data/joined_data_xxhash_tuned.csv'
with open(output_path, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in joined_data:
        csvwriter.writerow(row)

print(f"Read time: {read_time:.4f} seconds")
print(f"Join time: {join_time:.4f} seconds")
print(f"Total time: {read_time + join_time:.4f} seconds")
print(f"Joined data has been saved to {output_path}")


Read time: 2.7327 seconds
Join time: 0.7529 seconds
Total time: 3.4857 seconds
Joined data has been saved to /Users/shaiksaheer/Documents/College_Docs/DataBaseDesign/V3.0.1/dbgen/Data/joined_data_xxhash_tuned.csv


## Optimized FarmHash

In [11]:
import csv
import time
import farmhash

def read_csv_data_in_chunks(file_path, chunk_size=10000):
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        chunk = []
        for i, row in enumerate(reader):
            if i % chunk_size == 0 and i > 0:
                yield chunk
                chunk = []
            chunk.append(row)
        yield chunk

def farmhash_key(value):
    return farmhash.hash64(value)

def hash_join(orders_chunk, lineitem, hash_function):
    hash_table = {}
    for order in orders_chunk:
        hash_key = hash_function(order[0])
        hash_table[hash_key] = order

    joined_data = []
    for item in lineitem:
        hash_key = hash_function(item[0])
        if hash_key in hash_table:
            joined_data.append(hash_table[hash_key] + item)

    return joined_data

orders_path = '/Users/shaiksaheer/Documents/College_Docs/DataBaseDesign/V3.0.1/dbgen/Data/orders.csv'
lineitem_path = '/Users/shaiksaheer/Documents/College_Docs/DataBaseDesign/V3.0.1/dbgen/Data/lineitem.csv'

start_read_time = time.time()
lineitem_data = list(read_csv_data_in_chunks(lineitem_path, chunk_size=10000))[0]  # Read the first chunk for demonstration

# Perform the hash join in chunks
start_join_time = time.time()
joined_data = []
for orders_chunk in read_csv_data_in_chunks(orders_path, chunk_size=10000):
    joined_data.extend(hash_join(orders_chunk, lineitem_data, farmhash_key))
join_time = time.time() - start_join_time

output_path = '/Users/shaiksaheer/Documents/College_Docs/DataBaseDesign/V3.0.1/dbgen/Data/joined_data_farmhash_tuned.csv'
with open(output_path, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    for row in joined_data:
        csvwriter.writerow(row)

read_time = time.time() - start_read_time - join_time
print(f"Read time: {read_time:.4f} seconds")
print(f"Join time: {join_time:.4f} seconds")
print(f"Total time: {read_time + join_time:.4f} seconds")
print(f"Joined data has been saved to {output_path}")


Read time: 5.4128 seconds
Join time: 0.6477 seconds
Total time: 6.0605 seconds
Joined data has been saved to /Users/shaiksaheer/Documents/College_Docs/DataBaseDesign/V3.0.1/dbgen/Data/joined_data_farmhash_tuned.csv
